In [ ]:
%cd ..
%pwd

In [ ]:
from tradeforecast import Scrapper

ticker = 'AAPL'
scrapper = Scrapper(ticker)

exported_fname = scrapper.export_historic_data(interval='1d', period='max')

In [ ]:
from tradeforecast.augmentation import DataEntryPoint, RNNDataset, train_test_split
import polars as pl

data_entry = DataEntryPoint(exported_fname[ticker])

lf = data_entry.data.drop_nulls()
lf.collect()

In [ ]:
from torch.utils.data import DataLoader

dataset_kwargs = {'lf': lf,
                 'non_temporal': data_entry.non_temporal,
                 'temporal': data_entry.temporal,
                 'target': 'Close',
                 'look_back_len': 30,
                 'forecast_len': 5}

rnn_dataset = RNNDataset(**dataset_kwargs)

train_dataset, test_dataset = train_test_split(rnn_dataset, 0.05)
print(len(rnn_dataset), len(train_dataset), len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False)

In [ ]:
from tradeforecast import LSTM

lstm_kwargs = {'input_size': len(rnn_dataset.features),
              'hidden_size': 32,
              'n_LSTM': 4,
              'bidirectional': True,
              'fc_out_size':[],
              'output_size': rnn_dataset.forecast_len,
              'dropout': 0}

lstm_model = LSTM(**lstm_kwargs)
print(f"The device is set to ---> '{lstm_model.device}'")

lstm_model.load_model_state('.pth')

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from torch import Tensor

y: Tensor; y_preds: Tensor
y, y_preds = lstm_model.test_model(test_loader)

y_arr, y_preds_arr = y.cpu().numpy(), y_preds.cpu().numpy()
print(y_arr.shape, y_preds_arr.shape)

y_arr = rnn_dataset.inverse_transform(y_arr)
y_preds_arr = rnn_dataset.inverse_transform(y_preds_arr)
mean_absolute_percentage_error(y_arr, y_preds_arr)

In [ ]:
from tradeforecast.viz import plot_rnn_forecasts

plot_rnn_forecasts(y_arr, y_preds_arr)